In [1]:
import numpy as np
from PIL import Image
import cv2 #opencv
import io
import time
import pandas as pd
import numpy as np
from IPython.display import clear_output
from random import randint
import os

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

#keras imports
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD , Adam
from keras.callbacks import TensorBoard
from collections import deque
import random
import pickle
from io import BytesIO
import base64
import json

C:\Users\baker\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#path variables
game_url = "chrome://dino"
chrome_driver_path = "./chromedriver"
loss_file_path = "./objects/loss_df.csv"
actions_file_path = "./objects/actions_df.csv"
q_value_file_path = "./objects/q_values.csv"
scores_file_path = "./objects/scores_df.csv"

#scripts
#create id for canvas for faster selection from DOM
init_script = "document.getElementsByClassName('runner-canvas')[0].id = 'runner-canvas'"

#get image from canvas
getbase64Script = "canvasRunner = document.getElementById('runner-canvas'); \
return canvasRunner.toDataURL().substring(22)"


setting path

In [3]:
'''
* Game class: Selenium interfacing between the python and browser
* __init__():  Launch the broswer window using the attributes in chrome_options
* get_crashed() : return true if the agent as crashed on an obstacles. Gets javascript variable from game decribing the state
* get_playing(): true if game in progress, false is crashed or paused
* restart() : sends a signal to browser-javascript to restart the game
* press_up(): sends a single to press up get to the browser
* get_score(): gets current game score from javascript variables.
* pause(): pause the game
* resume(): resume a paused game if not crashed
* end(): close the browser and end the game
'''
class Game:
    def __init__(self,custom_config=True):
        chrome_options = Options()
        chrome_options.add_argument("disable-infobars")
        chrome_options.add_argument("--mute-audio")
        self._driver = webdriver.Chrome(executable_path = chrome_driver_path,chrome_options=chrome_options)
        self._driver.set_window_position(x=-10,y=0)
        self._driver.get('chrome://dino')
        self._driver.execute_script("Runner.config.ACCELERATION=0")
        self._driver.execute_script(init_script)
    def get_crashed(self):
        return self._driver.execute_script("return Runner.instance_.crashed")
    def get_playing(self):
        return self._driver.execute_script("return Runner.instance_.playing")
    def restart(self):
        self._driver.execute_script("Runner.instance_.restart()")
    def press_up(self):
        self._driver.find_element_by_tag_name("body").send_keys(Keys.ARROW_UP)
    def get_score(self):
        score_array = self._driver.execute_script("return Runner.instance_.distanceMeter.digits")
        score = ''.join(score_array) # the javascript object is of type array with score in the formate[1,0,0] which is 100.
        return int(score)
    def pause(self):
        return self._driver.execute_script("return Runner.instance_.stop()")
    def resume(self):
        return self._driver.execute_script("return Runner.instance_.play()")
    def end(self):
        self._driver.close()

銜接browser-javascript與python的模組，運行chrome並且連接至小恐龍遊戲的環境。
這裡可以得到狀態:撞毀、遊戲中、分數。指令:重啟遊戲、跳躍、暫停、恢復遊戲、結束。

In [4]:
class DinoAgent:
    def __init__(self,game): #takes game as input for taking actions
        self._game = game; 
        self.jump(); #to start the game, we need to jump once
    def is_running(self):
        return self._game.get_playing()
    def is_crashed(self):
        return self._game.get_crashed()
    def jump(self):
        self._game.press_up()
    def duck(self):
        self._game.press_down()

這裡為DinoAgent的模組，可知Agent是否撞毀，並有跳躍與蹲下兩種行為。我們在遊戲中只考慮跳躍與不跳躍兩種。

In [5]:
class Game_sate:
    def __init__(self,agent,game):
        self._agent = agent
        self._game = game
        self._display = show_img() #display the processed image on screen using openCV, implemented using python coroutine 
        self._display.__next__() # initiliaze the display coroutine 
    def get_state(self,actions):
        actions_df.loc[len(actions_df)] = actions[1] # storing actions in a dataframe
        score = self._game.get_score() 
        reward = 0.1
        is_over = False #game over
        if actions[1] == 1:
            self._agent.jump()
        image = grab_screen(self._game._driver) 
        self._display.send(image) #display the image on screen
        if self._agent.is_crashed():
            scores_df.loc[len(loss_df)] = score # log the score when game is over
            self._game.restart()
            reward = -1
            is_over = True
        return image, reward, is_over #return the Experience tuple

為了確認Game的state，這裡使用OpenCV的截圖來獲取環境的狀態(OpenCV中影像處理的速度有經過特別優化)，這裡的get_state方法為輸入一個action，然後回傳下一狀態的影像、reward(如果撞毀=-1，沒撞毀=0.1)、is_over(是否撞毀)。OpenCV平均可以達到5FPS的擷取速度，這對於每個跳Action後的State轉換已經足夠表示。

In [6]:
def save_obj(obj, name ):
    with open('objects/'+ name + '.pkl', 'wb') as f: #dump files into objects folder
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
def load_obj(name ):
    with open('objects/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

def grab_screen(_driver):
    image_b64 = _driver.execute_script(getbase64Script)
    screen = np.array(Image.open(BytesIO(base64.b64decode(image_b64))))
    image = process_img(screen)#processing image as required
    return image

def process_img(image):
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #RGB to Grey Scale
    image = image[:300, :500] #Crop Region of Interest(ROI)
    image = cv2.resize(image, (80,80))
    return  image

def show_img(graphs = False):
    """
    Show images in new window
    """
    while True:
        screen = (yield)
        window_title = "logs" if graphs else "game_play"
        cv2.namedWindow(window_title, cv2.WINDOW_NORMAL)        
        imS = cv2.resize(screen, (800, 400)) 
        cv2.imshow(window_title, screen)
        if (cv2.waitKey(1) & 0xFF == ord('q')):
            cv2.destroyAllWindows()
            break

儲存object方法，後面會儲存的資料有:行為，格式為(S,a,r,St+1,是否結束)、時間(為了辨認FPS的更新狀態)、epsilon(隨機行為的機率隨時間遞減的參數)，螢幕截圖(單純擷取chrome中的遊戲影像)，影像處理(只保留有興趣的影像區域，在小恐龍的遊戲中其實只在乎仙人掌與螢幕左邊界的距離。size為80*80)

In [7]:
#Intialize log structures from file if exists else create new
loss_df = pd.read_csv(loss_file_path) if os.path.isfile(loss_file_path) else pd.DataFrame(columns =['loss'])
scores_df = pd.read_csv(scores_file_path) if os.path.isfile(loss_file_path) else pd.DataFrame(columns = ['scores'])
actions_df = pd.read_csv(actions_file_path) if os.path.isfile(actions_file_path) else pd.DataFrame(columns = ['actions'])
q_values_df =pd.read_csv(actions_file_path) if os.path.isfile(q_value_file_path) else pd.DataFrame(columns = ['qvalues'])

使用DateFrame儲存loss,score,action,q_values。

In [8]:
#game parameters
ACTIONS = 2 # possible actions: jump, do nothing
GAMMA = 0.99 # decay rate of past observations original 0.99
OBSERVATION = 100. # timesteps to observe before training
EXPLORE = 100000  # frames over which to anneal epsilon
FINAL_EPSILON = 0.0001 # final value of epsilon
INITIAL_EPSILON = 0.1 # starting value of epsilon
REPLAY_MEMORY = 50000 # number of previous transitions to remember
BATCH = 16 # size of minibatch
FRAME_PER_ACTION = 1
LEARNING_RATE = 1e-4
img_rows , img_cols = 80,80
img_channels = 4 #We stack 4 frames

In [9]:
# training variables saved as checkpoints to filesystem to resume training from the same step
def init_cache():
    """initial variable caching, done only once"""
    save_obj(INITIAL_EPSILON,"epsilon")
    t = 0
    save_obj(t,"time")
    D = deque()
    save_obj(D,"D")

In [10]:
'''Call only once to init file structure
'''
#init_cache()

'Call only once to init file structure\n'

如果想要執行訓練，第一次需要執行init_cache()，他會創建所需的檔案。在這之前須建立object空白資料夾以存放。

In [11]:
def buildmodel():
    print("Now we build the model")
    model = Sequential()
    model.add(Conv2D(32, (8, 8), padding='same',strides=(4, 4),input_shape=(img_cols,img_rows,img_channels)))  #80*80*4
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (4, 4),strides=(2, 2),  padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3),strides=(1, 1),  padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(ACTIONS))
    adam = Adam(lr=LEARNING_RATE)
    model.compile(loss='mse',optimizer=adam)
    
    #create model file if not present
    if not os.path.isfile(loss_file_path):
        model.save_weights('model.h5')
    print("We finish building the model")
    return model

模型輸入為80*80*4，即每四張影像為一組State，再經過幾次捲積與池化，再展平成512的特徵向量，最後輸出(0,1)代表不跳與跳，這裡的類神經網路用來替代傳統的Q-table，由於影像為格式的state很難窮舉到Q-table中，因此使用類神經網路，好處是可以舉一反三，State的pixel值即使不盡相同，但是仍可以訓練出其特性。這個方法即為DQN，為一種off-policy learning。

In [12]:
''' 
main training module
Parameters:
* model => Keras Model to be trained
* game_state => Game State module with access to game environment and dino
* observe => flag to indicate wherther the model is to be trained(weight updates), else just play
'''
def trainNetwork(model,game_state,observe=False):
    last_time = time.time()
    # store the previous observations in replay memory
    D = load_obj("D") #load from file system
    # get the first state by doing nothing
    do_nothing = np.zeros(ACTIONS)
    do_nothing[0] =1 #0 => do nothing,
                     #1=> jump
    
    x_t, r_0, terminal = game_state.get_state(do_nothing) # get next step after performing the action
    

    s_t = np.stack((x_t, x_t, x_t, x_t), axis=2) # stack 4 images to create placeholder input
    

    
    s_t = s_t.reshape(1, s_t.shape[0], s_t.shape[1], s_t.shape[2])  #1*20*40*4
    
    initial_state = s_t 

    if observe :
        OBSERVE = 999999999    #We keep observe, never train
        epsilon = FINAL_EPSILON
        print ("Now we load weight")
        model.load_weights("model.h5")
        adam = Adam(lr=LEARNING_RATE)
        model.compile(loss='mse',optimizer=adam)
        print ("Weight load successfully")    
    else:                       #We go to training mode
        OBSERVE = OBSERVATION
        #epsilon = load_obj("epsilon") 
        epsilon = 0
        model.load_weights("model.h5")
        adam = Adam(lr=LEARNING_RATE)
        model.compile(loss='mse',optimizer=adam)

    t = load_obj("time") # resume from the previous time step stored in file system
    while (True): #endless running
        
        loss = 0
        Q_sa = 0
        action_index = 0
        r_t = 0 #reward at 4
        a_t = np.zeros([ACTIONS]) # action at t
        
        #choose an action epsilon greedy
        if t % FRAME_PER_ACTION == 0: #parameter to skip frames for actions
            if  random.random() <= epsilon: #randomly explore an action
                print("----------Random Action----------")
                action_index = random.randrange(ACTIONS)
                a_t[0] = 1
            else: # predict the output
                q = model.predict(s_t)       #input a stack of 4 images, get the prediction
                max_Q = np.argmax(q)         # chosing index with maximum q value
                action_index = max_Q 
                a_t[action_index] = 1        # o=> do nothing, 1=> jump
                
        #We reduced the epsilon (exploration parameter) gradually
        #if epsilon >= FINAL_EPSILON and t > OBSERVE:
            #epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / EXPLORE 
            #epsilon=0
        #run the selected action and observed next state and reward
        x_t1, r_t, terminal = game_state.get_state(a_t)
        print('fps: {0}'.format(1 / (time.time()-last_time))) # helpful for measuring frame rate
        last_time = time.time()
        x_t1 = x_t1.reshape(1, x_t1.shape[0], x_t1.shape[1], 1) #1x20x40x1
        s_t1 = np.append(x_t1, s_t[:, :, :, :3], axis=3) # append the new image to input stack and remove the first one
        
        
        # store the transition in D
        D.append((s_t, action_index, r_t, s_t1, terminal))
        if len(D) > REPLAY_MEMORY:
            D.popleft()

        #only train if done observing
        if t > OBSERVE: 
            
            #sample a minibatch to train on
            minibatch = random.sample(D, BATCH)
            inputs = np.zeros((BATCH, s_t.shape[1], s_t.shape[2], s_t.shape[3]))   #32, 20, 40, 4
            targets = np.zeros((inputs.shape[0], ACTIONS))                         #32, 2

            #Now we do the experience replay
            for i in range(0, len(minibatch)):
                state_t = minibatch[i][0]    # 4D stack of images
                action_t = minibatch[i][1]   #This is action index
                reward_t = minibatch[i][2]   #reward at state_t due to action_t
                state_t1 = minibatch[i][3]   #next state
                terminal = minibatch[i][4]   #wheather the agent died or survided due the action
                

                inputs[i:i + 1] = state_t    

                targets[i] = model.predict(state_t)  # predicted q values
                Q_sa = model.predict(state_t1)      #predict q values for next step
                
                if terminal:
                    targets[i, action_t] = reward_t # if terminated, only equals reward
                else:
                    targets[i, action_t] = reward_t + GAMMA * np.max(Q_sa)

            loss += model.train_on_batch(inputs, targets)
            loss_df.loc[len(loss_df)] = loss
            q_values_df.loc[len(q_values_df)] = np.max(Q_sa)
        s_t = initial_state if terminal else s_t1 #reset game to initial frame if terminate
        t = t + 1
        
        # save progress every 1000 iterations
        if t % 20000 == 0:
            print("Now we save model")
            game_state._game.pause() #pause game while saving to filesystem
            model.save_weights("model.h5", overwrite=True)
            save_obj(D,"D") #saving episodes
            save_obj(t,"time") #caching time steps
            save_obj(epsilon,"epsilon") #cache epsilon to avoid repeated randomness in actions
            loss_df.to_csv("./objects/loss_df.csv",index=False)
            scores_df.to_csv("./objects/scores_df.csv",index=False)
            actions_df.to_csv("./objects/actions_df.csv",index=False)
            q_values_df.to_csv(q_value_file_path,index=False)
            with open("model.json", "w") as outfile:
                json.dump(model.to_json(), outfile)
            clear_output()
            game_state._game.resume()
        # print info
        state = ""
        if t <= OBSERVE:
            state = "observe"
        elif t > OBSERVE and t <= OBSERVE + EXPLORE:
            state = "explore"
        else:
            state = "train"

        print("TIMESTEP", t, "/ STATE", state,             "/ EPSILON", epsilon, "/ ACTION", action_index, "/ REWARD", r_t,             "/ Q_MAX " , np.max(Q_sa), "/ Loss ", loss)

    print("Episode finished!")
    print("************************")


這裡寫下訓練的過程，我們先觀察100次，然後到累積10萬筆之前都是EXPLORE狀態，但仍會訓練神經網路，這是因為epsilon(隨機動作的機率)在10萬次以後才會收斂到最低值，後面的狀態我們稱為訓練狀態。
訓練方式是這樣的:使用當前state預測下一步的action並且概率地(取決於epsilon)執行，這時候觀察St+1與reward並記錄下來，然後從replay buffer中隨機取出32筆資料出來訓練，得到32組(S,A,r,St+1,terminal)，如果恐龍在此State,S採取Action,A，陣亡了，那麼reward就不更新，但如果沒有陣亡，則reward=reward+GAMMA*max(Q_sa)，意思是此State的Action會讓你的恐龍活著，所以是好棒棒的，所以多給一點reward，然後在拿其S,A,r進行supervised learning，Ptrue為更新過的reward，在進行反向傳播法更新參數，如此一來網路就會更像理想的Q_table，

In [ ]:
#main function
def playGame(observe=False):
    game = Game()
    dino = DinoAgent(game)
    game_state = Game_sate(dino,game)    
    model = buildmodel()
    try:
        trainNetwork(model,game_state,observe=observe)
    except StopIteration:
        game.end()

In [ ]:
playGame(observe=False);

Now we build the model
We finish building the model
fps: 0.215493308272021
TIMESTEP 320001 / STATE train / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  10.396174 / Loss  3.8856799602508545
fps: 0.9123853754833237
TIMESTEP 320002 / STATE train / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  10.204363 / Loss  0.020749880000948906
fps: 5.637641770903059
TIMESTEP 320003 / STATE train / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  10.164232 / Loss  2.414555788040161
fps: 7.135900642252563
TIMESTEP 320004 / STATE train / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  10.083686 / Loss  0.04089697450399399
fps: 6.101222919966892
TIMESTEP 320005 / STATE train / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  9.904004 / Loss  0.44432443380355835
fps: 6.535464087158136
TIMESTEP 320006 / STATE train / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  10.267053 / Loss  0.019154895097017288
fps: 5.99106690111171
TIMESTEP 320007 / STATE train / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  10.038898 / Loss  0.0298473387956619

TIMESTEP 320061 / STATE train / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  10.288724 / Loss  0.023942384868860245
fps: 3.8435986818713324
TIMESTEP 320062 / STATE train / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  10.0591345 / Loss  0.022996719926595688
fps: 5.827024654002073
TIMESTEP 320063 / STATE train / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  10.033935 / Loss  0.029853232204914093
fps: 7.053008073208751
TIMESTEP 320064 / STATE train / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  9.934183 / Loss  0.013776391744613647
fps: 7.138147347640362
TIMESTEP 320065 / STATE train / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  10.105944 / Loss  0.1146804541349411
fps: 6.234380248822034
TIMESTEP 320066 / STATE train / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  9.899192 / Loss  0.5389309525489807
fps: 5.958063613415893
TIMESTEP 320067 / STATE train / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  9.770458 / Loss  0.9702993035316467
fps: 6.063893246203112
TIMESTEP 320068 / STATE train / EPSILON 0 / ACTI

TIMESTEP 320122 / STATE train / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  9.244808 / Loss  0.048689231276512146
fps: 5.951764966312671
TIMESTEP 320123 / STATE train / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  9.975339 / Loss  0.023605480790138245
fps: 6.01588342022791
TIMESTEP 320124 / STATE train / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  10.194627 / Loss  0.024496495723724365
fps: 6.132050286770264
TIMESTEP 320125 / STATE train / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  10.071356 / Loss  0.004357010126113892
fps: 5.995897239428244
TIMESTEP 320126 / STATE train / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  9.975541 / Loss  0.00271231634542346
fps: 6.06713871390032
TIMESTEP 320127 / STATE train / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  9.70193 / Loss  0.8360134959220886
fps: 7.223612646778199
TIMESTEP 320128 / STATE train / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  10.169338 / Loss  0.017084412276744843
fps: 7.1122204662456845
TIMESTEP 320129 / STATE train / EPSILON 0 / ACTION

TIMESTEP 320184 / STATE train / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  10.238222 / Loss  0.0213266983628273
fps: 6.931049811037648
TIMESTEP 320185 / STATE train / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  10.235319 / Loss  0.012403502129018307
fps: 7.194655002358592
TIMESTEP 320186 / STATE train / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  10.276084 / Loss  0.02407602034509182
fps: 6.986978242654127
TIMESTEP 320187 / STATE train / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  10.327284 / Loss  0.018533051013946533
fps: 7.125571881683987
TIMESTEP 320188 / STATE train / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  10.365924 / Loss  0.3269802927970886
fps: 7.043757473163785
TIMESTEP 320189 / STATE train / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  -1.6743178 / Loss  0.04758094251155853
fps: 7.115224696895415
TIMESTEP 320190 / STATE train / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  10.675167 / Loss  0.010274015367031097
fps: 6.999302456753988
TIMESTEP 320191 / STATE train / EPSILON 0 / AC

fps: 5.995931524963368
TIMESTEP 320246 / STATE train / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  10.917196 / Loss  1.1954554319381714
fps: 5.996480146855936
TIMESTEP 320247 / STATE train / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  11.419979 / Loss  0.7940719723701477
fps: 6.066217349801857
TIMESTEP 320248 / STATE train / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  11.457732 / Loss  0.022206194698810577
fps: 6.025822672058961
TIMESTEP 320249 / STATE train / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  11.106314 / Loss  0.01621951535344124
fps: 7.175245316071111
TIMESTEP 320250 / STATE train / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  10.944967 / Loss  0.01592765934765339
fps: 6.012649445224921
TIMESTEP 320251 / STATE train / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  11.115246 / Loss  0.010860549286007881
fps: 6.007215583827811
TIMESTEP 320252 / STATE train / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  10.693668 / Loss  2.0125315189361572
fps: 5.887494402793901
TIMESTEP 320253 / STATE tr

...

TIMESTEP 330052 / STATE train / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  8.819834 / Loss  0.021684784442186356
fps: 6.723300301677017
TIMESTEP 330053 / STATE train / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  6.826252 / Loss  0.05277600884437561
fps: 5.967922960467154
TIMESTEP 330054 / STATE train / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  9.351079 / Loss  0.04468538612127304
fps: 6.040227824221085
TIMESTEP 330055 / STATE train / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  8.771182 / Loss  0.03191152215003967
fps: 6.02754592904279
TIMESTEP 330056 / STATE train / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  9.177843 / Loss  0.01789683662354946
fps: 5.969723710354172
TIMESTEP 330057 / STATE train / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  9.101183 / Loss  0.19233955442905426
fps: 3.1476386166403008
TIMESTEP 330058 / STATE train / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  9.220584 / Loss  0.019538328051567078
fps: 4.947204159407794
TIMESTEP 330059 / STATE train / EPSILON 0 / ACTION 1 

fps: 6.672898921022403
TIMESTEP 330114 / STATE train / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  8.913259 / Loss  4.658478736877441
fps: 5.4164501448930995
TIMESTEP 330115 / STATE train / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  8.360769 / Loss  0.04071692377328873
fps: 5.50596896192309
TIMESTEP 330116 / STATE train / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  8.563083 / Loss  0.015464022755622864
fps: 6.6916679297058845
TIMESTEP 330117 / STATE train / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  8.696931 / Loss  1.268826961517334
fps: 6.686472750692671
TIMESTEP 330118 / STATE train / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  8.52865 / Loss  0.20418670773506165
fps: 6.654298247701546
TIMESTEP 330119 / STATE train / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  7.8499675 / Loss  0.04401964321732521
fps: 6.682467203477053
TIMESTEP 330120 / STATE train / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  8.427872 / Loss  0.6630389094352722
fps: 5.485245575765185
TIMESTEP 330121 / STATE train / EPS

TIMESTEP 330175 / STATE train / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  7.8447356 / Loss  0.037387602031230927
fps: 6.826922231789276
TIMESTEP 330176 / STATE train / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  7.943412 / Loss  0.011704158037900925
fps: 7.159824585062418
TIMESTEP 330177 / STATE train / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  8.21617 / Loss  0.40238305926322937
fps: 7.049368896956252
TIMESTEP 330178 / STATE train / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  8.067004 / Loss  0.5738698840141296
fps: 7.191891618469853
TIMESTEP 330179 / STATE train / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  8.006933 / Loss  0.030372820794582367
fps: 6.003423735994458
TIMESTEP 330180 / STATE train / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  8.067943 / Loss  0.10649637132883072
fps: 5.973473093547721
TIMESTEP 330181 / STATE train / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  8.3510895 / Loss  0.02828269451856613
fps: 5.970768983291885
TIMESTEP 330182 / STATE train / EPSILON 0 / ACTION 1

fps: 5.989330226075831
TIMESTEP 330237 / STATE train / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  8.593185 / Loss  0.29272520542144775
fps: 7.109363373329785
TIMESTEP 330238 / STATE train / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  8.726121 / Loss  0.059877660125494
fps: 6.116695881956617
TIMESTEP 330239 / STATE train / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  8.005333 / Loss  0.07673390209674835
fps: 6.018049971734146
TIMESTEP 330240 / STATE train / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  8.744798 / Loss  0.08198775351047516
fps: 6.1436550193714705
TIMESTEP 330241 / STATE train / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  8.7127495 / Loss  0.041050516068935394
fps: 5.996763069250874
TIMESTEP 330242 / STATE train / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  9.300002 / Loss  0.05103690177202225
fps: 7.1302112728731615
TIMESTEP 330243 / STATE train / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  8.643298 / Loss  0.020849719643592834
fps: 5.997534811493155
TIMESTEP 330244 / STATE train

fps: 5.983998173828683
TIMESTEP 330299 / STATE train / EPSILON 0 / ACTION 0 / REWARD -1 / Q_MAX  8.602729 / Loss  0.03061351738870144
fps: 7.002001619325059
TIMESTEP 330300 / STATE train / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  8.4819145 / Loss  0.03552568703889847
fps: 7.195593061576705
TIMESTEP 330301 / STATE train / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  8.201715 / Loss  0.06982887536287308
fps: 7.241609475219917
TIMESTEP 330302 / STATE train / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  8.099216 / Loss  0.4117426872253418
fps: 6.007723246513653
TIMESTEP 330303 / STATE train / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  9.12439 / Loss  0.0450669601559639
fps: 7.002328932013549
TIMESTEP 330304 / STATE train / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  8.199802 / Loss  0.10721258819103241
fps: 6.899479533356582
TIMESTEP 330305 / STATE train / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  8.702607 / Loss  0.013387154787778854
fps: 7.032773636220351
TIMESTEP 330306 / STATE train / EP